# 장고 Star Rating 달기 

### 일단 재성이의 Recommender-System의 셋팅이 되어있다고 가정 

####  1. base.html 헤드 태그 안에 Rate Yo! CDN을 추가하자

http://rateyo.fundoocode.ninja/#installation 

#### 2. product_detail.html 에 rateYo 객체? 불러오기 

* product_detail.html 에다가 rateYo 객체를 먼저 추가해준다

#### 3. View 추가해주기

"""AJAX로 POST로 오는 요청을 처리한다. 우선 이미 이 유저가 이 product에 매긴 rating이 있었는지 체크하고 (Vote 모델 내에 인스턴스가 있는지 체크), 만약 있었다면 새로운 별점으로 수정을 해주고, 처음 Rating 매기는 것이라면 새롭게 Vote 모델 안에 인스턴스를 추가한다. 그리고 다시 ProductView로 요청을 전달한다."""


In [1]:
def newRating(request):

    username = request.user
    product = request.POST['product']
    try:
        vote = Vote.objects.get(user=username.id, product = product)
        vote.score = request.POST['scoring']
        vote.save()
        return redirect('myrecommend.views.ProductView', pk=product)
    except:
        vote = Vote()
        vote.user = username
        vote.product_id = product
        vote.site_id = 1
        vote.score = request.POST['scoring']
        vote.save()
    
    return redirect('myrecommend.views.ProductView', pk=product)
    

#### 4. ProductView도 약간의 수정이 필요하다. 

원래는 특정 제품을 자세히 보기 위해 'object'만 product_detail.html로 전달해줬지만 이젠 우린 그 특정제품에 대한 별점까지 같이 보여줘야 하기 때문에 별점에 대한 정보를 같이 던져준다.

먼저 Vote 모델을 뒤져서 유저가 그 product에 대해 rating 매긴적이 있다면 그 score를 voteRating 변수에 담고, 한번도 별점을 매긴적이 없다면 voteRating에 0을 담아서 product_detail.html에 함께 전달한다.

In [2]:
def ProductView(request,pk=None):
    object1=Product.objects.get(id=int(pk))
    username= request.user
    try:
        vote = Vote.objects.get(user=username.id, product = object1.id)
        print username.id
        voteRating = vote.score
    except:
        voteRating = 0

    return render(request, 'product_detail.html',{'object':object1, 'voteRating': voteRating})
    

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-2-a98c63e79ff6>, line 6)

#### 5. ajax로 post를 해주기 위한 url 추가

urls.py에 

url(r'^newRating/$', 'myrecommend.views.newRating', name='new_rating'), 추가

#### 6. product_detail.html 에 rateYo 스크립트와 ajax 스크립트 추가 

* rateYo를 동작하게 만드는  (최초 불러왔을때 별점을 초기화해주는 onInit fuction과 새롭게 눌렀을 때 수정하거나 추가해주는 onSet function 그리고 별점을 반개까지 줄 수 있는 옵션인 halfstar: True,) 그리고..


* 가장 중요한.. 우리가 Productview에서 받아온 voteRating을 rateYo의 변수 rating에 할당해준다!!

product_detail.html 에서 컨텐츠블록이 끝나기 전에 {% endblock %} 위에다가 아래 스크립트를 추가 